In [1]:
import torch
from torchvision.datasets import MNIST, CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
from src.sde import VE_SDE, VP_SDE, SubVP_SDE
from src.unet import Unet

In [3]:
from src.unet import Unet

device = torch.device('cuda:1')

model = Unet(
    dim=28,
    dim_mults=(1, 2, 4),
    channels=1
).to(device)
model.load_state_dict(torch.load('ckpts/sub_vp_sde_mnist.pth'))

sde = SubVP_SDE(eps=1e-3, rescale=True).to(device)

In [4]:
samples = sde.predictor_corrector_sample(model, (8, 1, 28, 28), device, n_steps=500, n_lang_steps=3, snr=0.16)

100%|██████████| 500/500 [01:01<00:00,  8.17it/s]


In [8]:
from torchvision.transforms.functional import to_pil_image

samples = torch.clamp(samples, 0, 1)
to_pil_image(samples[0])